In [5]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from database.weather import Weather
from transformer.date_transformer import DateTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
import pickle

In [6]:
## Loading Constants
start = "2008-01-01"
end = datetime.now().strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("portfolio")
market = Market()
weather = Weather()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
correlations = market.retrieve_data("coint")
market.close()
reload = True
year = datetime.now().year
week = datetime.now().isocalendar()[1]
model_range = range(len(tickers["Symbol"]))
quarter = math.ceil((datetime.now().month - 1) // 3) + 1

In [7]:
quarter

2

In [8]:
gap = 5
week_gap = int(gap/5)
training_years = 1
sec.connect()
strat_db.connect()
classification = strat_db.retrieve_data("dataset_pdr_week_classification")
regression = strat_db.retrieve_data("dataset_pdr_week_regression")
strat_db.drop_table("weekly_models")
strat_db.drop_table("weekly_model_features")
for col in regression.columns:
    if -99999 == regression[col].min():
        regression.drop(col,axis=1,inplace=True)
for col in classification.columns:
    if -99999 == classification[col].min():
        classification.drop(col,axis=1,inplace=True)
try:
    for i in tqdm(model_range):
        try:
            ticker = tickers.iloc[i]["Symbol"].replace(".","-")
            if ticker in regression.columns:
                price = strat_db.retrieve_price_data("pdr_prices",ticker)
                mt = ModelTransformer(ticker,start,end,week_gap)
                mr = ModelPreprocessor(ticker)
                prot = ProductTransformer(ticker,start,end)
                ## applying cointegration / correlations results
                correlations["same"] = [abs(x) for x in correlations["corr"]]
                tc = correlations[(correlations["t1"]==ticker) | (correlations["t2"]==ticker)]
                tc = tc[tc["same"] >= 0.8]
                relevant_tickers = tc["t1"].unique().tolist()
                relevant_tickers.extend(tc["t2"].unique().tolist())
                rt = list(set(relevant_tickers))
                regression_tickers = []
                for relevant_ticker in rt:
                    if relevant_ticker in regression.columns:
                        regression_tickers.append(relevant_ticker)
                regression_tickers.extend(["year","quarter","week"])
                ticker_regression = regression[regression_tickers]
                ## regression_model
                first = ticker_regression[(ticker_regression["year"] == year - training_years) & (ticker_regression["quarter"] == quarter)].index.values.tolist()[0]
                last = ticker_regression[(ticker_regression["year"] == year) & (ticker_regression["quarter"] == quarter)].index.values.tolist()[0]
                rqpd = ticker_regression.iloc[first:last-1]
                rqpd["y"] = rqpd[ticker]
                rqpd["y"] = rqpd["y"].shift(-week_gap)
                rqpd = rqpd[:-week_gap]
                qpd = mr.day_trade_preprocess_regression(rqpd.copy(),ticker,True)
                q2c = qpd["X"].columns
                stuff = []
                for col in q2c:
                    stuff.append({"ticker":ticker,"model":"regression","feature":col})
                strat_db.store_data("weekly_model_features",pd.DataFrame(stuff))
                rpr = sp.regression(qpd,ranked=False,tf=False,deep=False)
                ## classification_model
                classification_tickers = []
                for relevant_ticker in rt:
                    if relevant_ticker in classification.columns:
                        classification_tickers.append(relevant_ticker)
                classification_tickers.extend(["year","quarter","week"])
                ticker_classification = classification[classification_tickers]
                first = ticker_classification[(ticker_classification["year"] == year - training_years) & (ticker_classification["quarter"] == quarter)].index.values.tolist()[0]
                last = ticker_classification[(ticker_classification["year"] == year) & (ticker_classification["quarter"] == quarter)].index.values.tolist()[0]
                cqpd = ticker_classification.iloc[first:last-1]
                cqpd["y"] = cqpd[ticker]
                cqpd["y"] = cqpd["y"].shift(-week_gap)
                cqpd = cqpd[:-week_gap]
                qpd = mr.day_trade_preprocess_classify(cqpd.copy(),ticker)
                q2c = qpd["X"].columns
                stuff = []
                for col in q2c:
                    stuff.append({"ticker":ticker,"model":"classification","feature":col})
                strat_db.store_data("weekly_model_features",pd.DataFrame(stuff))
                cpr = sp.classification(qpd,tf=False,deep=False)
                price_results = pd.DataFrame([cpr,rpr])
                price_results["ticker"] = ticker
                price_results["year"] = year
                price_results["quarter"] = quarter
                price_results["model"] = [pickle.dumps(x) for x in price_results["model"]]
                strat_db.store_data("weekly_models",price_results)
        except Exception as e:
            message = {"status":"weekly modeling","ticker":ticker,"year":str(year),"week":str(week),"message":str(e)}
            print(message)
except Exception as e:
    print(year,quarter,str(e))
sec.close()
strat_db.close()

  6%|██████████▌                                                                                                                                                                        | 30/506 [01:31<19:14,  2.43s/it]

{'status': 'weekly modeling', 'ticker': 'AMC', 'year': '2021', 'week': '13', 'message': "'AMC'"}


 21%|█████████████████████████████████████▎                                                                                                                                            | 106/506 [05:36<18:45,  2.81s/it]

{'status': 'weekly modeling', 'ticker': 'CNP', 'year': '2021', 'week': '13', 'message': "'CNP'"}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 506/506 [25:08<00:00,  2.98s/it]


In [9]:
price_results

,api,model,score,model_type,ticker,year,quarter
2,skl,"b""\x80\x04\x95\xf2\x08\x00\x00\x00\x00\x00\x00...",0.714286,classification,ZTS,2021,2
0,xgb,b'\x80\x04\x95\x0f#\x00\x00\x00\x00\x00\x00\x8...,0.021333,regression,ZTS,2021,2
